In [1]:
#load in necessary packages 
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.axes as axs
import matplotlib.dates as mdates
from scipy.io import loadmat
from os.path import dirname, join as pjoin
from datetime import datetime, timedelta


In [3]:
#load in data
filepath=r"W:\PROJECTS\Greenland_ADCP\2023_Data\2023_07_22_Day_2_Transects\104505_20230722T124546UTC_Data_15pptCorrection\104505_20230722T124546UTC.mat"

In [5]:
##This part was written by AI but converts the subtructures from the .mat file to a python dictionary 
import scipy.io as spio
import numpy as np

def loadmat_recursive(filename):
    """
    Recursively loads a .mat file, converting MATLAB objects to Python dictionaries.
    """
    def _check_keys(d):
        """Checks if entries in dictionary are mat-objects and converts them accordingly."""
        for key in d:
            if isinstance(d[key], spio.matlab.mat_struct):
                d[key] = _todict(d[key])
            elif isinstance(d[key], np.ndarray):
                d[key] = _tolist(d[key])
        return d

    def _todict(matobj):
        """Converts a mat-object to a dictionary."""
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        """Converts a numpy array to a list, handling nested mat-objects."""
        elem_list = []
        for elem in ndarray:
            if isinstance(elem, spio.matlab.mat_struct):
                elem_list.append(_todict(elem))
            elif isinstance(elem, np.ndarray):
                elem_list.append(_tolist(elem))
            else:
                elem_list.append(elem)
        return elem_list
    
    # Use the function by calling it instead of direct spio.loadmat
    data = spio.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

In [7]:
# Example usage of the function
data = loadmat_recursive(filepath)
# Now you can access data more naturally:
# result = mat_data['main_structure_name']['sub_structure_1']['data_field_name']


In [9]:
data['A']['Nav']['timestamp']

[739089.5317916671,
 739089.5317986115,
 739089.5318101855,
 739089.5318217597,
 739089.5318333337,
 739089.5318449078,
 739089.5318564818,
 739089.5318680559,
 739089.53187963,
 739089.5318912041,
 739089.5319027782,
 739089.5319143522,
 739089.5319259263,
 739089.5319375003,
 739089.5319490745,
 739089.5319606485,
 739089.5319722256,
 739089.5319837966,
 739089.5319953708,
 739089.5320069448,
 739089.5320185189,
 739089.532030093,
 739089.532041667,
 739089.5320532412,
 739089.5320648152,
 739089.5320763893,
 739089.5320879633,
 739089.5320995374,
 739089.5321111114,
 739089.5321226856,
 739089.5321342596,
 739089.5321458337,
 739089.5321574077,
 739089.5321689819,
 739089.532180556,
 739089.53219213,
 739089.5322037063,
 739089.5322152781,
 739089.5322268523,
 739089.5322384263,
 739089.5322500004,
 739089.5322615744,
 739089.5322731485,
 739089.5322847225,
 739089.5322962967,
 739089.5323078707,
 739089.5323194448,
 739089.5323310189,
 739089.532342593,
 739089.5323541671,
 739089.

In [33]:
current=pd.DataFrame(data['A']['Wat']['vMag_ms'])#.fillna(0)
print(current.shape)
current=current.iloc[:,:-1]

(69, 3599)


In [35]:
depth_nonan=pd.DataFrame(data['A']['Nav']['depth']) #.fillna(0) # fill NaNs with zeros for plotting
depthmax= depth_nonan.max().max() # grab maximum depth for plotting
print(depthmax)
print(depth_nonan.shape)

46.32592405558589
(3599, 4)


In [37]:
#Function to convert matlab timestamp to regular datetime in UTC
def dtnum_dttime_adcp(datenum_array):
    dates = []
    for datenum in datenum_array:
        python_datetime = datetime.fromordinal((int(datenum))) + timedelta(days=(datenum) % 1) - timedelta(days=366)
        dates.append(python_datetime)
    return dates

In [27]:
timestamp = dtnum_dttime_adcp(data['A']['Nav']['timestamp'])
print(timestamp)
print(type(timestamp))

[datetime.datetime(2023, 7, 22, 12, 45, 46, 800038), datetime.datetime(2023, 7, 22, 12, 45, 47, 400035), datetime.datetime(2023, 7, 22, 12, 45, 48, 400029), datetime.datetime(2023, 7, 22, 12, 45, 49, 400034), datetime.datetime(2023, 7, 22, 12, 45, 50, 400028), datetime.datetime(2023, 7, 22, 12, 45, 51, 400033), datetime.datetime(2023, 7, 22, 12, 45, 52, 400028), datetime.datetime(2023, 7, 22, 12, 45, 53, 400032), datetime.datetime(2023, 7, 22, 12, 45, 54, 400037), datetime.datetime(2023, 7, 22, 12, 45, 55, 400031), datetime.datetime(2023, 7, 22, 12, 45, 56, 400036), datetime.datetime(2023, 7, 22, 12, 45, 57, 400031), datetime.datetime(2023, 7, 22, 12, 45, 58, 400035), datetime.datetime(2023, 7, 22, 12, 45, 59, 400030), datetime.datetime(2023, 7, 22, 12, 46, 0, 400034), datetime.datetime(2023, 7, 22, 12, 46, 1, 400029), datetime.datetime(2023, 7, 22, 12, 46, 2, 400295), datetime.datetime(2023, 7, 22, 12, 46, 3, 400028), datetime.datetime(2023, 7, 22, 12, 46, 4, 400033), datetime.datetim

In [39]:
# create a meshgrid of time and depth
x,y=np.meshgrid(timestamp,depth_nonan,current)
z=current
print(z.shape,x.shape,y.shape)

MemoryError: Unable to allocate 93.6 TiB for an array with shape (14396, 3599, 248262) and data type object

In [ ]:
# Plot Energy Spectra over time per cell
plt.figure(figsize=(15,6))
plt.pcolormesh(
    x,y,z, cmap='jet', shading='gouraud'
)
#plt.ylim(top=0.3)
plt.ylabel("Depth (m)")
im1=plt.colorbar()
im1.ax.set_ylabel("Velocity magnitude m/s")
plt.show()